<div class="alert alert-block alert-warning">
    <b>Warning</b>
    
Before running this notebook, make sure that you have Gromacs installed and that you have sourced `GMXRC` to get access to GROMACS, e.g. `source /usr/local/gromacs/bin/GMXRC`.
</div>

# Running a Molecular Dynamics (MD) simulation of a protein-ligand complex


In this notebook we run an MD simulation of benzene bound to T4-lysozyme L99A using Gromacs.![image](assets/t4lyso.png)

## On the MD protocol

The Gromacs MD protocol allows the user to run an MD simulation in solvent of e.g. a small molecule, a protein, or a protein-ligand complex. Running an MD simulations can be useful for a variety of things, such as pre-equilibration of a protein-ligand complex prior to running free energy calculations or for getting insights into the general dynamics of the system of interest.

The MD protocol uses different software tools in a series of steps and provides multiple outputs for the user:

| **Step**                                                        | **Software used**                  |
|:----------------------------------------------------------------|:-----------------------------------|
| 1. Input handling using gufe                                    | OpenFE, Gufe, RDKit                |
| 2. Parameterization using OpenMMForceFields & OpenFF            | OpenFE - OpenMMForceFields - OpenFF|
| 3. Creating of Gromacs input files (.gro, .top) via Interchange | OpenFE - OpenFF                    |
| 4. Minimization                                                 | GROMACS                            |
| 5. NVT equilibration                                            | GROMACS                            |
| 6. NPT MD                                                       | GROMACS                            |

## 1. Defining the ChemicalSystem

`ChemicalSystems` are containers which define the various components which exist in a system of interest. 
Here, we will be passing the `SmallMoleculeComponent` for benzene, a `ProteinComponent` generated from a PDB file, and a `SolventComponent` which will contain the necessary information for OpenMM’s Modeller class to add water and 0.15 M NaCl around the solute.

In [1]:
import openfe_gromacs
from openfe_gromacs import ChemicalSystem, ProteinComponent, SmallMoleculeComponent, SolventComponent
from openff.units import unit

# Define the ligand we are interested in
ligand = SmallMoleculeComponent.from_sdf_file('assets/benzene.sdf')

# Define the solvent environment and protein structure
solvent = SolventComponent(ion_concentration=0.15 * unit.molar)
protein = ProteinComponent.from_pdb_file('assets/t4_lysozyme.pdb', name='t4-lysozyme')

# create the ChemicalSystem
system = ChemicalSystem({'ligand': ligand, 'protein': protein, 'solvent': solvent}, name=f"{ligand.name}_{protein.name}")

## 2. Defining the MD simulation settings

There are various different parameters which can be set to determine how the MD simulation will take place. To allow for maximum user flexibility, these are defined as a series of settings objects which control the following:

| **Setting**                    | **Description**                                                |
|:------------------------------|:----------------------------------------------------------------|
|  `protocol_repeats`           | How many repeats of the MD protocol should be run.              |
| `simulation_settings_em`      | Energy minimization simulation settings, e.g. `nsteps`.         |
| `simulation_settings_nvt`     | NVT MD simulation settings, e.g. `nsteps`.                      |
| `simulation_settings_npt`     | NPT MD simulation settings, e.g. `nsteps`.                      |
| `output_settings_em`          | Energy minimization outputs settings, e.g. file names and write frequencies. |
| `output_settings_nvt`         | NVT MD output settings, e.g. file names and write frequencies.  |
| `output_settings_npt`         | NPT MD output settings, e.g. file names and write frequencies.  |
| `forcefield_settings`         | Settings controlling how the system is parameterized.           |
| `partial_charge_settings`     | Settings controlling how small molecule partial charges are assigned. |
| `solvation_settings`          | Settings controlling how the system should be solvated.         |
| `thermo_settings`             | Settings controlling thermodynamic parameters e.g. the `temperature` and the `pressure` of the system. |
| `top`                         | Name of the Gromacs topology .top file that should be written (using Interchange). |
| `gro`                         | Name of the Gromacs cordinate .gro file that should be written (using Interchange). |

The easiest way to access and change settings is by first importing the default settings, printing them and then changing the settings according to the user's needs.

In [2]:
from openfe_gromacs.protocols.gromacs_md.md_methods import GromacsMDProtocol
from openff.units import unit

settings = GromacsMDProtocol.default_settings()
settings.simulation_settings_em.nsteps = 10 # setting the number of minimization steps to 10
settings.simulation_settings_nvt.nsteps = 10 # setting the number of NVT MD steps to 10
settings.simulation_settings_npt.nsteps = 10 # setting the number of NVT MD steps to 10
settings.solvation_settings.box_shape = 'dodecahedron'

In [3]:
settings

{'engine_settings': {'bonded': 'auto',
                     'nb': 'auto',
                     'ntomp': 1,
                     'pme': 'auto',
                     'pmefft': 'auto',
                     'update': 'auto'},
 'forcefield_settings': {'constraints': None,
                         'forcefields': ['amber/ff14SB.xml',
                                         'amber/tip3p_standard.xml',
                                         'amber/tip3p_HFE_multivalent.xml',
                                         'amber/phosaa10.xml'],
                         'hydrogen_mass': 3.0,
                         'nonbonded_cutoff': <Quantity(1.0, 'nanometer')>,
                         'nonbonded_method': 'PME',
                         'rigid_water': True,
                         'small_molecule_forcefield': 'openff-2.1.1'},
 'gro': 'system.gro',
 'integrator_settings': {'barostat_frequency': <Quantity(25, 'timestep')>,
                         'constraint_tolerance': 1e-06,
                  

## 3.  Creating a `Protocol`

The actual simulation is performed by a [`Protocol`](https://docs.openfree.energy/en/stable/guide/models/execution.html#protocols-and-the-execution-model). 

With the `Settings` inspected and adjusted, we can provide these to the `Protocol`. Here, the Gromacs-based MD Protocol is named `GromacsMDProtocol`.

In [4]:
# Creating the Protocol
from openfe_gromacs.protocols.gromacs_md.md_methods import GromacsMDProtocol
protocol = GromacsMDProtocol(settings=settings)

## 4. Running the MD simulation
Here we will show you how to run the MD simulation using our Python API.

The MD simulation can be run by executing the `ProtocolDAG`. The `ProtocolDAG` is created using the `protocol.create()` method and requires as input the `ChemicalSystem`. 

Note: we use the ``shared_basedir`` and ``scratch_basedir`` argument of ``execute_DAG`` in order to set the directory where the simulation files are written to.

In [5]:
import gufe
import pathlib

# Creating the Protocol
protocol = GromacsMDProtocol(settings=settings)
# Creating the Protocol DAG
dag = protocol.create(stateA=system, stateB=system, mapping=None)
workdir = pathlib.Path('')
# Running the MD simulations
dagres = gufe.protocols.execute_DAG(
    dag,
    shared_basedir=workdir,
    scratch_basedir=workdir,
    keep_shared=True, # set this to True to save the outputs
    n_retries=3
)

/home/ialibay/github/openfe-gromacs/openfe_gromacs/protocols/gromacs_md/md_methods.py:797: UserWarning: Environment variable INTERCHANGE_EXPERIMENTAL=1 is set for using the interchange functionality 'from_openmm' which is not well tested yet.
  warnings.warn(war)
                :-) GROMACS - gmx grompp, 2024.3-conda_forge (-:

Executable:   /home/ialibay/software/mambaforge/install/envs/openfe_gromacs/bin.AVX2_256/gmx
Data prefix:  /home/ialibay/software/mambaforge/install/envs/openfe_gromacs
Working dir:  /home/ialibay/github/openfe-gromacs/examples
Command line:
  gmx grompp -f shared_GromacsMDSetupUnit-299a978e93804186b4364a2666955e11_attempt_0/em.mdp -c shared_GromacsMDSetupUnit-299a978e93804186b4364a2666955e11_attempt_0/system.gro -p shared_GromacsMDSetupUnit-299a978e93804186b4364a2666955e11_attempt_0/system.top -o shared_GromacsMDRunUnit-1158654926f04d45bd07ea3e203f294d_attempt_0/em.tpr

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group re

Setting the LD random seed to -846332307

Generated 3457135 of the 3457135 non-bonded parameter combinations

Generated 3457135 of the 3457135 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'MOL0'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL2'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL7874'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL7895'

turning H bonds into constraints...
Analysing residue names:
There are:   154    Protein residues
There are:     1      Other residues
There are:  7872      Water residues
There are:    50        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

The largest distance between excluded atoms is 0.418 nm between atom 98 and 105
Calculating four


NOTE: Thread affinity was not set.

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =           10

Energy minimization reached the maximum number of steps before the forces
reached the requested precision Fmax < 10.

writing lowest energy coordinates.

Steepest Descents did not converge to Fmax < 10 in 11 steps.
Potential Energy  = -3.1863700e+05
Maximum force     =  1.6095507e+04 on atom 2551
Norm of force     =  3.9362164e+02

GROMACS reminds you: "I Calculate My Birthright" (P.J. Harvey)

                :-) GROMACS - gmx grompp, 2024.3-conda_forge (-:

Executable:   /home/ialibay/software/mambaforge/install/envs/openfe_gromacs/bin.AVX2_256/gmx
Data prefix:  /home/ialibay/software/mambaforge/install/envs/openfe_gromacs
Working dir:  /home/ialibay/github/openfe-gromacs/examples
Command line:
  gmx grompp -f shared_GromacsMDSetupUnit-299a978e93804186b4364a2666955e11_attempt_0/nvt.mdp -c shared_GromacsMDRunUnit-1158654926f04d45bd07ea3e203f294d_attempt_0/

Setting the LD random seed to -949390596

Generated 3457135 of the 3457135 non-bonded parameter combinations

Generated 3457135 of the 3457135 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'MOL0'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL2'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL7874'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL7895'

turning H bonds into constraints...

Setting gen_seed to 213381799

Velocities were taken from a Maxwell distribution at 298.15 K
Analysing residue names:
There are:   154    Protein residues
There are:     1      Other residues
There are:  7872      Water residues
There are:    50        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Th

Changing nstlist from 10 to 80, rlist from 1.2 to 1.314


Update groups can not be used for this system because there are three or more consecutively coupled constraints

Using 1 MPI thread
Using 1 OpenMP thread 


NOTE: Thread affinity was not set.
starting mdrun 'FOO'
10 steps,      0.0 ps.

Writing final coordinates.

NOTE: 1 % of the run time was spent in domain decomposition,
      14 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:        0.234        0.234      100.0
                 (ns/day)    (hour/ns)
Performance:        8.123        2.954

GROMACS reminds you: "Not everyone is capable of madness; and of those lucky enough to be capable, not many have the courage for it." (August Strindberg)

                :-) GROMACS - gmx grompp, 2024.3-conda_forge (-:

Executable:   /home/ialibay/software/mambaforge/install/envs/openfe_gromacs/bin.AVX2_256

Setting the LD random seed to 2138832851

Generated 3457135 of the 3457135 non-bonded parameter combinations

Generated 3457135 of the 3457135 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'MOL0'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL2'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL7874'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MOL7895'

turning H bonds into constraints...
Analysing residue names:
There are:   154    Protein residues
There are:     1      Other residues
There are:  7872      Water residues
There are:    50        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

The largest distance between excluded atoms is 0.425 nm between atom 1682 and 1692

Determining 

Changing nstlist from 10 to 80, rlist from 1.2 to 1.314


Update groups can not be used for this system because there are three or more consecutively coupled constraints

Using 1 MPI thread
Using 1 OpenMP thread 


NOTE: Thread affinity was not set.
starting mdrun 'FOO'
10 steps,      0.0 ps.

Writing final coordinates.

NOTE: 1 % of the run time was spent in domain decomposition,
      13 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:        0.245        0.245      100.0
                 (ns/day)    (hour/ns)
Performance:        7.757        3.094

GROMACS reminds you: "Between equal rights, force decides." (Karl Marx)



Following files were created for the setup of the MD run:

In [6]:
!ls shared_GromacsMDSetupUnit-1349c37dbb574349adc4370ec7a9bd15_attempt_0/

ls: cannot access 'shared_GromacsMDSetupUnit-1349c37dbb574349adc4370ec7a9bd15_attempt_0/': No such file or directory


Following files were created by the energy minimization, NVT, and NPT MD simulations:

In [7]:
!ls shared_GromacsMDRunUnit-15d6dbfe6fa3424885094b1b02a51a29_attempt_0

ls: cannot access 'shared_GromacsMDRunUnit-15d6dbfe6fa3424885094b1b02a51a29_attempt_0': No such file or directory
